## **Import the TensorFlow library**

Bisogna togliere le librerie che non servono o sono duplicate rispetto a quelle che già ci sono nel file

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dropout, Dense, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers

import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from PIL import Image

import numpy as np
import pandas as pd
import seaborn as sns
import random 
import csv 
import sys
import os
import re
from random import seed

import time
import cv2
import glob
from keras import applications
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score

from numpy.ma.core import size

import torchvision.models as models
from torchvision import transforms

# LOAD TRAINED MODEL AND FEATURES

Qui vanno chiaramente messi i nuovi path dove sono messi i txt di features, le loro labels e il file h5

In [ ]:
MODEL_PATH = ROOT_PATH + "/Saved_models/Backup_models/Last_trained_model/"

In [ ]:
features = np.loadtxt(MODEL_PATH + 'Original_model/ll_features_10.txt')

In [ ]:
labels_features = np.loadtxt(MODEL_PATH + 'Original_model/ll_labels_features_10.txt')
labels_features = labels_features.astype(int)

In [ ]:
model = keras.models.load_model(MODEL_PATH + 'Original_model/mnist_cnn.h5')

In [ ]:
model_frozen = keras.models.load_model(MODEL_PATH + 'Frozen_model/omv_mnist_cnn.h5')

In [ ]:
model.summary()

# K-MEAN FUNCTIONS

In [ ]:
def create_k_mean(data, number_of_clusters):
    # n_jobs is set to -1 to use all available CPU cores. This makes a big difference on an 8-core CPU
    # especially when the data size gets much bigger. #perfMatters

    k = KMeans(n_clusters=number_of_clusters, n_init=100)
    # k = KMeans(n_clusters=number_of_clusters, n_init=20, max_iter=500)

    # Let's do some timings to see how long it takes to train.
    start = time.time()

    # Train it up
    k.fit(data)

    # Stop the timing
    end = time.time()

    # And see how long that took
    # print("Training took {} seconds".format(end - start))

    return k

In [ ]:
def cluster_label_count(clusters, labels):
    count = {}

    # Get unique clusters and labels
    unique_clusters = list(set(clusters))
    unique_labels = list(set(labels))

    # Create counter for each cluster/label combination and set it to 0
    for cluster in unique_clusters:
        count[cluster] = {}

        for label in unique_labels:
            count[cluster][label] = 0

    # Let's count
    for i in range(len(clusters)):
        count[clusters[i]][labels[i]] += 1

    cluster_df = pd.DataFrame(count)

    return cluster_df, count

In [ ]:
def k_mean_clustering(features_x_test, features_init, y_test, labels_features_init, n_cluster, batch_size):

  # creo due vettori vuoti uno per le label lungo (n_features) l'altro per le features lungo (n_features,512)
  features = np.zeros((int(n_cluster*len(labels_features_init)/10),512))
  labels_features = np.zeros(int(n_cluster*len(labels_features_init)/10))
  k = 0 #inizializzo un counter

  #passo il vettore salvato nel file e mi tiro fuori solo i numeri che mi interessano. Se ho 6 cluster, mi interessano solo gli 0, 1, 2, 3, 4 e 5.
  for i in range(0,len(features_init)-1):
      if labels_features_init[i] < n_cluster:
        features[k,:] = features_init[i,:]
        labels_features[k] = labels_features_init[i]
        k += 1
  
  labels_features = labels_features.astype(int)

  # aggiungo la feature che di cui ci interessa trovare la pseudo label
  features = np.concatenate((features, features_x_test))
  labels_features = np.append(labels_features, y_test)
  n_count = np.zeros((batch_size, n_cluster)).astype(int) #inizializzo un vettore che ci permette di contare per ogni cluster quali numeri sono stati inseriti

  # viene fatto il clustering
  k_mean = create_k_mean(features ,n_cluster)

  # qui viene riempito il vettore conteggio. Se l'immagine che ci interessa classificare è stata inserita nel cluster 0, il ciclo guarderà 
  # tutte le labels delle immagini che sono state messe in quel cluster e la label che compare più volte sarà la pseudo_label della nostra immagine
  for j in range(1, batch_size+1):
    for i in range(0,len(k_mean.labels_) - batch_size):
        if k_mean.labels_[i] == k_mean.labels_[-j]:
         if labels_features[i] == 0:
           n_count[-j,0] +=1
         if labels_features[i] == 1:
           n_count[-j,1] +=1
         if labels_features[i] == 2:
           n_count[-j,2] +=1
         if labels_features[i] == 3:
           n_count[-j,3] +=1
         if labels_features[i] == 4:
           n_count[-j,4] +=1
         if labels_features[i] == 5:
           n_count[-j,5] +=1
         if labels_features[i] == 6:
           n_count[-j,6] +=1
         if labels_features[i] == 7:
           n_count[-j,7] +=1
         if labels_features[i] == 8:
           n_count[-j,8] +=1
         if labels_features[i] == 9:
           n_count[-j,9] +=1
  
  pseudo_label = np.zeros(batch_size)
  err = 0      

  # We extract the label for the new element
  for i in range(0,batch_size):
    pseudo_label[i]  =   np.argmax(n_count[i,:])  
    if pseudo_label[i]!= labels_features[i+int(n_cluster*len(labels_features_init)/10)]:
      err += 1

  # Delete the last feature and the last label
  features = np.delete(features, len(features)-1, 0)

  #elimino label che abbiamo concatenato
  labels_features = np.delete(labels_features, len(labels_features)-1, 0)

  return pseudo_label, err

# FUNCTIONS

In [ ]:
def softmax(array):
    
    if(len(array.shape)==2):
        array = array[0]
        
    size    = len(array)
    ret_ary = np.zeros([len(array)])
    m       = array[0]
    sum_val = 0

    for i in range(0, size):
        if(m<array[i]):
            m = array[i]

    for i in range(0, size):
        sum_val += np.exp(array[i] - m)

    constant = m + np.log(sum_val)
    for i in range(0, size):
        ret_ary[i] = np.exp(array[i] - constant)
        
    return ret_ary

In [ ]:
class Custom_Layer(object):
    def __init__(self, model):

        # Related to the layer
        self.ML_frozen = keras.models.Sequential(model.layers[:-1])  # extract the last layer from the original model
        self.ML_frozen.compile()
        
        self.W = np.array(model.layers[-1].get_weights()[0])    # extract the weights from the last layer
        self.b = np.array(model.layers[-1].get_weights()[1])    # extract the biases from the last layer
               
        self.W_2 = np.zeros(self.W.shape)
        self.b_2 = np.zeros(self.b.shape)
        
        self.label     = [0,1,2,3,4,5]              
        self.std_label = [0,1,2,3,4,5,6,7,8,9]
        
        self.l_rate = 0                                         # learning rate that changes depending on the algorithm        

        self.batch_size = 0
        
        # Related to the results fo the model
        self.conf_matr = np.zeros((10,10))    # container for the confusion matrix       
        self.macro_avrg_precision = 0       
        self.macro_avrg_recall = 0
        self.macro_avrg_F1score = 0
        
        self.title = ''       # title that will be displayed on plots
        self.filename = ''    # name of the files to be saved (plots, charts, conf matrix)
        
        
    # Function that is used for the prediction of the model saved in this class
    def predict(self, x):
        mat_prod = np.array(np.matmul(x, self.W) + self.b)
        return softmax(mat_prod) # othwerwise do it with keras|also remove np.array()| tf.nn.softmax(mat_prod) 

In [ ]:
def digitToSoftmax(current_label, known_labels):
    ret_ary = np.zeros(len(known_labels))

    known_labels_2 = [0,1,2,3,4,5]
                       
    for i in range(0, len(known_labels)):
        if(current_label == known_labels_2[i]):
            ret_ary[i] = 1

    return ret_ary  


In [ ]:
def NumberToSoftmax(current_label, known_labels):
    ret_ary = np.zeros(len(known_labels))
                       
    for i in range(0, len(known_labels)):
        if(current_label == known_labels[i]):
            ret_ary[i] = 1

    return ret_ary

In [ ]:
def CheckLabelKnown(model, current_label):
    
    found = 0
    
    for i in range(0, len(model.label)):
        if(current_label == model.label[i]):
            found = 1
        
        
    # If the label is not known
    if(found==0):
        print(f'\n\n    New digit detected -> digit \033[1m{current_label}\033[0m \n')

        model.label.append(current_label)   # Add new digit to label
                
        # Increase weights and biases dimensions
        model.W = np.hstack((model.W, np.zeros([model.W.shape[0],1])))
        model.b = np.hstack((model.b, np.zeros([1])))
        
        model.W_2 = np.hstack((model.W_2, np.zeros([model.W.shape[0],1])))
        model.b_2 = np.hstack((model.b_2, np.zeros([1])))

In [ ]:
def trainOneEpoch_OL(model, x_test, y_test, features, labels_features, batch_size):
       
    cntr = 1
    learn_rate = model.l_rate
    
    test_samples = x_test.shape[0]
    
    n_cluster = 10
    err_tot = 0
    cntr_clus = 0
    reminder = test_samples%batch_size
    max_iter = int(test_samples//batch_size)

    features_images = np.zeros(test_samples)
        
    # FEATURE EXTRACTION

    print('**********************************\n Performing features extraction \n')

    for i in range(0, test_samples):
        y_ML   = model.ML_frozen.predict(x_test[i].reshape((1,28,28,1)))

        if i == 0:
          features_images = np.copy(y_ML)       
        else:
          features_images = np.concatenate((features_images, y_ML))

    # CLUSTERING

    print('**********************************\n Performing clustering\n')

    # Pseudo-labels
    for i in range(0, max_iter):
      
        pseudo_label, err = k_mean_clustering(features_images[i*batch_size:i*batch_size+batch_size], features, y_test[i*batch_size:i*batch_size+batch_size], labels_features, n_cluster, batch_size)
        err_tot += err
        pseudo_label = pseudo_label.astype(int)

        if i == 0:
          pseudo_labels = np.copy(pseudo_label)      
        else:
          pseudo_labels = np.append(pseudo_labels, pseudo_label)

        print(f"\r    Currently at {np.round(np.round(cntr_clus/test_samples*batch_size,4)*100,2)}% of dataset", end="")
        print("\n")
        cntr_clus +=1
    
    # Pseudo-labels for last batch
    
    if reminder != 0: 
        pseudo_label, err = k_mean_clustering(features_images[max_iter*batch_size:test_samples], features, y_test[max_iter*batch_size:test_samples], labels_features, n_cluster, reminder)
        err_tot = err_tot + err
        pseudo_labels = np.append(pseudo_labels, pseudo_label)

    # Check errors in clustering

    #print("The error in clustering is: ")
    #print(int(err_tot/test_samples*100))
    #print("%")
    #print("\n")

    # ONLINE-LEARNING

    print('**********************************\n Performing training with OL\n')

    for i in range(0, test_samples):

        CheckLabelKnown(model, pseudo_labels[i])
    
        y_true_soft = NumberToSoftmax(pseudo_labels[i], model.label)
               
        # Prediction
        
        y_pred = model.predict(features_images[i,:])
        
        # Backpropagation
        cost = y_pred-y_true_soft
        
        for j in range(0,model.W.shape[0]):

            # Update weights
            dW = np.multiply(cost, features_images[i,j]*learn_rate)
            model.W[j,:] = model.W[j,:]-dW

        # Update biases
        db      = np.multiply(cost, learn_rate)
        model.b = model.b-db
        
    return y_pred

# TRAIN THE MODEL ON THE DIGITS 6-9 (OL METHOD)

In [ ]:
Model_OL = Custom_Layer(model)
Model_OL.title      = 'OL'
Model_OL.filename   = 'OL'
Model_OL.l_rate     = 0.01
Model_OL.batch_size = 8

batch_size = 1

prediction = trainOneEpoch_OL(Model_OL, '''qui le feautere da analizzare''', '''qui le labels delle feautere da analizzare''', features, labels_features, batch_size)